### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [48]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [49]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [50]:
# cell for work
# 'The number of movies in the dataset': 
# 'The number of ratings in the dataset':
print(len(movies))

print(len(reviews))

35479
863866


In [67]:
# cell for work

# 'The number of different genres':
# print(np.unique(movies['genre']))
genres = []
for genre in movies.genre:
#     print(genre)
    try:
        genres.extend(genre.split('|'))
    except AttributeError:
        pass
len(np.unique(genres))

28

In [6]:
# more work

# 'The number of unique users in the dataset':
print(len(reviews['user_id'].unique()))

67353


In [7]:
# and still more

# 'The number missing ratings in the reviews dataset':
reviews['rating'].isnull().sum()

0

In [8]:
# # and even more - open additional cells as necessary
# 'The average rating given across all ratings':
print(reviews['rating'].mean())
# 'The minimum rating given across all ratings':
print(reviews['rating'].min())
# 'The maximum rating given across all ratings':
print(reviews['rating'].max())

7.31587769399
0
10


In [9]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 35479,
'The number of ratings in the dataset': 863866,
'The number of different genres': 2737,
'The number of unique users in the dataset': 67353,
'The number missing ratings in the reviews dataset': 0,
'The average rating given across all ratings': 7.31587769399,
'The minimum rating given across all ratings': 0,
'The maximum rating given across all ratings': 10
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea
# Solution
# a = reviews.user_id.nunique()
# b = reviews.rating.max()
# c = np.round(reviews.rating.mean(), 0)
# d = movies.shape[0]
# e = reviews.rating.min()
# f = int(reviews.rating.isnull().mean()*reviews.shape[0])
# h = reviews.shape[0]
# i = len(genres)

#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [40]:
# cell for work
movies.head()
# Pull the date from the title and create new column
movies['Date'] = movies['movie'].str.extract(r'\((.*?)\)', expand=False)#.astype(int)
# movies['movie'][movies['movie'].index('(')+1 : movies['movie'].index(')')]
# movies['movie'].iloc[0][movies['movie'].index('(') : movies['movie'].index(')')+1]
# movies['movie'].iloc[0][movies['movie'].iloc[0].index('(')+1 : movies['movie'].iloc[0].index(')')]
movies.head()


,movie_id,movie,genre,Date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


In [41]:
# and another
# Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
movies['century'] = movies['Date'].str[0:2] + '00'
cent_dummy = pd.get_dummies(movies['century'])
# cent_dummy
movies = pd.concat([movies, cent_dummy], axis=1)
movies = movies.drop(['century'], axis=1)
movies.head()

,movie_id,movie,genre,Date,1800,1900,2000
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [37]:
# and a few more below, which you can use as necessary
# Dummy column the genre with 1's and 0's
# movies['genre'] = pd.get_dummies(movies['genre'], dummy_na=True)

# genre_dummy = pd.get_dummies(movies['genre'])
# movies = pd.concat([movies, genre_dummy], axis=1)
# movies = movies.drop(['genre'], axis=1)

movies.head()

,movie_id,movie,Date,1800,1900,2000,0,1
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),1894,1,0,0,0,1
1,0000010,La sortie des usines Lumière (1895),1895,1,0,0,0,1
2,0000012,The Arrival of a Train (1896),1896,1,0,0,0,1
3,25,The Oxford and Cambridge University Boat Race ...,1895,1,0,0,0,1
4,0000091,Le manoir du diable (1896),1896,1,0,0,0,1


In [ ]:
# movies['genre'].nunique()
# print(movies['genre'].iloc[0])
def split_genre(genre):
    if genre:
        return 1
    else:
        return 0
        
movies['genre_split'] = movies['genre'].str.split('|')
for genre in genres:    
    movies[genre] = movies['genre_split'].apply(split_genre)
    
movies.head()

In [114]:
# Reviews
# Create a date out of time stamp
reviews.head()
import datetime

def covert_timestamp(timestamp):
    date_time = datetime.datetime.fromtimestamp(timestamp)
    return date_time

reviews['Date_New'] = reviews['timestamp'].astype(int).apply(covert_timestamp)
# datetime.datetime.fromtimestamp(reviews['timestamp'].astype(int).iloc[0])
# reviews['timestamp'].apply(covert_timestamp)
reviews.head()

,user_id,movie_id,rating,timestamp,Date_New
0,1,0114508,8,1381006850,2013-10-05 21:00:50
1,2,0208092,5,1586466072,2020-04-09 21:01:12
2,2,0358273,9,1579057827,2020-01-15 03:10:27
3,2,10039344,5,1578603053,2020-01-09 20:50:53
4,2,6751668,9,1578955697,2020-01-13 22:48:17


In [94]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year